In [ ]:
from flask import Flask,render_template,url_for,request
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import pandas as pd 
import pickle
import numpy as np 
import nltk
from nltk.corpus import stopwords 
import re
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.externals import joblib

porter_stemmer = nltk.stem.porter.PorterStemmer()

#spilts the sentences into words
def porter_tokenizer(text, stemmer=porter_stemmer):
    lower_txt = text.lower()
    tokens = nltk.wordpunct_tokenize(lower_txt)
    stems = [porter_stemmer.stem(t) for t in tokens]
    no_punct = [s for s in stems if re.match('^[a-zA-Z]+$', s) is not None]
    return no_punct

stop_words = set(stopwords.words('english'))

tfidf_vectorizer = TfidfVectorizer(stop_words='english',
                                   encoding='utf-8',
                                   decode_error='replace',
                                   strip_accents='unicode',
                                   analyzer='word',
                                   tokenizer=porter_tokenizer,
                                   ngram_range=(1,2),
                                   binary=False)


def vectorizeData():
    df= pd.read_csv('CleanData.csv')
    le = LabelEncoder()
    le.fit(df['Label'])

    df_labels = pd.DataFrame(np.array(le.transform(df['Label'])))
    
    skf = StratifiedKFold(n_splits = 5)
    
    for trn_indx, tst_indx in skf.split(df['Content'],df_labels):
        skf_X_train, skf_X_test = df['Content'].iloc[trn_indx], df['Content'].iloc[tst_indx]
        skf_Y_train, skf_Y_test = df_labels.iloc[trn_indx], df_labels.iloc[tst_indx]
        
    # Fit and transform the training data for tfidf
    tfidf_train = tfidf_vectorizer.fit_transform(skf_X_train)

    # Transform the test set 
    tfidf_test = tfidf_vectorizer.transform(skf_X_test)
    
    return tfidf_train,tfidf_test,skf_Y_train,skf_Y_test

#Generate the training and testing dataset
X_train, X_test ,Y_train, Y_test  = vectorizeData()

#Using the SVM Model
from sklearn import svm
clf = svm.LinearSVC()
clf.fit(X_train,Y_train)
clf.score(X_test,Y_test)
  

app = Flask(__name__)

@app.route('/')
def home():
	return render_template('home.html')

@app.route('/predict',methods=['POST'])
def predict():
    #SVM Classifier
#     from sklearn import svm
#     clf = svm.LinearSVC()
#     clf.fit(X_train,Y_train)
#     clf.score(X_test,Y_test)
# #     #Alternative Usage of Saved Model
#     ytb_model = open("naivebayes_spam_model.pkl","rb")
#     clf = joblib.load(ytb_model)

    if request.method == 'POST':
        comment = request.form['comment']
        data = [comment]
        vect = tfidf_vectorizer.transform(data).toarray()
        my_prediction = clf.predict(vect)
        print(my_prediction)
    return render_template('result.html',prediction = my_prediction)


if __name__ == '__main__':
    app.run(debug=False)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:300: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['abov', 'afterward', 'alon', 'alreadi', 'alway', 'ani', 'anoth', 'anyon', 'anyth', 'anywher', 'becam', 'becaus', 'becom', 'befor', 'besid', 'cri', 'describ', 'dure', 'els', 'elsewher', 'empti', 'everi', 'everyon', 'everyth', 'everywher', 'fifti', 'formerli', 'forti', 'ha', 'henc', 'hereaft', 'herebi', 'hi', 'howev', 'hundr', 'inde', 'latterli', 'mani', 'meanwhil', 'moreov', 'mostli', 'nobodi', 'noon', 'noth', 'nowher', 'onc', 'onli', 'otherwis', 'ourselv', 'perhap', 'pleas', 'seriou', 'sever', 'sinc', 'sincer', 'sixti', 'someon', 'someth', 'sometim', 'somewher', 'themselv', 'thenc', 'thereaft', 'therebi', 'therefor', 'thi', 'thu', 'togeth', 'twelv', 'twenti', 'veri', 'wa', 'whatev', 'whenc', 'whenev', 'wherea', 'whereaft', 'wherebi', 'wherev', 'whi', 'yourselv'] not in stop_words.
  'stop

[0]


127.0.0.1 - - [25/Apr/2019 12:30:39] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [25/Apr/2019 12:30:39] "GET /static/css/styles.css HTTP/1.1" 404 -


[1]


127.0.0.1 - - [25/Apr/2019 12:31:09] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [25/Apr/2019 12:31:09] "GET /static/css/styles.css HTTP/1.1" 404 -


...

[0]


127.0.0.1 - - [25/Apr/2019 12:35:26] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [25/Apr/2019 12:35:26] "GET /static/css/styles.css HTTP/1.1" 404 -


[0]


127.0.0.1 - - [25/Apr/2019 12:40:52] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [25/Apr/2019 12:40:52] "GET /static/css/styles.css HTTP/1.1" 404 -


[0]
